## 1. 라이브러리 로드

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

## 2. 대표폰트/클러스터별 폰트 로드

In [2]:
fonts = pd.read_csv('대표폰트 및 클러스터별 폰트(유클리드거리).csv')
fonts

,0,1,2,3
0,GOYANGDEOGYANG B,KoPubWorld Dotum Medium,KoPubWorld Batang Light,netmarbleM
1,10X10,12롯데마트드림Light,Arita-buriB,20950870_경기천년바탕TTF_Bold
2,10X10Bold,12롯데마트드림Medium,Arita-buriL,20950871_경기천년바탕TTF_Regular
3,12롯데마트드림Bold,20950914_빛고을광주_Light_TTF,Arita-buriM,20950938_Batang_TTF
4,12롯데마트행복Bold,20956072_SuncheonR_TTF,Arita-buriSB,20956073_SuncheonB_TTF
...,...,...,...,...
73,이순신돋움체B,NaN,NaN,NaN
74,이순신돋움체M,NaN,NaN,NaN
75,한수원_한돋음_B,NaN,NaN,NaN
76,한수원_한돋음_R,NaN,NaN,NaN


In [3]:
repres_fonts = fonts.loc[[0]]
fonts_foreach_cluster = fonts.iloc[1:, :]
fonts_foreach_cluster

,0,1,2,3
1,10X10,12롯데마트드림Light,Arita-buriB,20950870_경기천년바탕TTF_Bold
2,10X10Bold,12롯데마트드림Medium,Arita-buriL,20950871_경기천년바탕TTF_Regular
3,12롯데마트드림Bold,20950914_빛고을광주_Light_TTF,Arita-buriM,20950938_Batang_TTF
4,12롯데마트행복Bold,20956072_SuncheonR_TTF,Arita-buriSB,20956073_SuncheonB_TTF
5,12롯데마트행복Light,Atomy-Light,BareunBatangB,20958935_UTOIMAGE_유토이미지고딕_R_TTF
...,...,...,...,...
73,이순신돋움체B,NaN,NaN,NaN
74,이순신돋움체M,NaN,NaN,NaN
75,한수원_한돋음_B,NaN,NaN,NaN
76,한수원_한돋음_R,NaN,NaN,NaN


## 3-1. 텍스트 렌더링 함수

In [4]:
from PIL import Image, ImageDraw, ImageFont
import os

def make_image(message, rendering_time, font_list, img_save_path):

  # font setting
  font_color = 'rgb(0, 0, 0)'
  font_size = 60
  
  # image size
  width = len(message) * font_size + 10
  height = font_size + 35
  bg_color = 'rgb(255, 255, 255)'
  x_margin = 5
  y_margin = 5
  
  # 렌더링 결과 파일 저장할 폴더 생성
  img_save_path = img_save_path + message + '(' + str(rendering_time) + ')/'
  os.mkdir(img_save_path)
  
  for font in font_list:
    image =Image.new('RGB', (width, height), color = bg_color)
    font_name = font.replace('.ttf', '').replace('.TTF', '')
    font = ImageFont.truetype('C:/Users/SM/dir_/FontProject/_석사코드정리_학제간연구용/1. 폰트파일-_이미지파일 변환 코드/ttf_to_jpg/ttf_standard_font(183)/' + font, font_size)
    draw = ImageDraw.Draw(image)      
    draw.text((x_margin, y_margin), message, font=font, fill=font_color)
    
    # save file
    image.save(img_save_path + font_name + '.png')
  
  return img_save_path

## 3-2. 렌더링 이미지의 공백 제거 함수

In [5]:
def trim(img_path):
  img_path = img_path
  img_list = os.listdir(img_path)
  
  for img_name in img_list:
    img = Image.open(img_path + img_name)
    pixels = img.load()  
    
    xlist = []
    ylist = []
    for y in range(0, img.size[1]):
      for x in range(0, img.size[0]):
        if pixels[x, y] != (255, 255, 255):
          xlist.append(x)
          ylist.append(y)
    left = min(xlist)
    right = max(xlist)
    top = min(ylist)
    bottom = max(ylist)
    
    trimed_img = img.crop((left, top, right, bottom))
    trimed_img.save(img_path + img_name)

## 4. 입력이미지 특징벡터 추출 

In [6]:
import sys
sys.path.append("C:/Users/SM/dir_/FontProject/_석사코드정리_학제간연구용/3. 유사폰트추천코드/img2vec/img2vec_pytorch")  # Adds higher directory to python modules path.
from img_to_vec import Img2Vec

img2vec =Img2Vec(model="inception")

test_img_input_path = 'C:/Users/SM/Desktop/2022-2 졸업프로젝트/Similar font recommendation/훈민정음.png'
test_filename = os.fsdecode(test_img_input_path)
test_img = Image.open(os.path.join(test_img_input_path, test_filename)).convert('RGB')
test_img_vector = img2vec.get_vec(test_img)
    
test_img_vector_df = pd.DataFrame(test_img_vector).transpose()
test_img_vector_df

AdaptiveAvgPool2d(output_size=(1, 1))


,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,1.134893,0.185287,0.011392,0.0,0.010423,0.032565,0.096117,0.665791,0.0,0.005898,...,1.090696,1.475597,0.002462,0.044556,1.387439,0.0,0.504765,0.239854,0.075132,0.723508


## 5. 1차 유사도 비교

### 5.1 대표폰트로 텍스트 렌더링

In [22]:
font_list = os.listdir('C:/Users/SM/dir_/FontProject/_석사코드정리_학제간연구용/1. 폰트파일-_이미지파일 변환 코드/ttf_to_jpg/ttf_standard_font(183)')

repres_font_list = []
for font_idx in repres_fonts:
  repres_font_list.append(repres_fonts[font_idx].values[0] + '.ttf')

msg = "훈민정음"
rendering_time = 1
img_save_path = "C:/Users/SM/Desktop/2022-2 졸업프로젝트/Similar font recommendation/"
trim_img_path = make_image(msg, rendering_time, repres_font_list, img_save_path)

trim(trim_img_path)


C:/Users/SM/Desktop/2022-2 졸업프로젝트/Similar font recommendation/훈민정음(1)/


### 5.2 렌더링한 텍스트 이미지의 특징벡터 추출

In [9]:
input_path = 'C:/Users/SM/Desktop/2022-2 졸업프로젝트/Similar font recommendation/훈민정음(1)/'
list_pics1 = []
filenames1 = []
for file1 in os.listdir(input_path):
  filename1 = os.fsdecode(file1)
  img1 = Image.open(os.path.join(input_path, filename1)).convert('RGB')
  list_pics1.append(img1)
  filenames1.append(filename1)
    
vectors1 = img2vec.get_vec(list_pics1)
pics1 = {}
for i1, vec1 in enumerate(vectors1):
  pics1[filenames1[i1]] = vec1
    
rendering_1st_vector_df = pd.DataFrame(pics1).transpose()
rendering_1st_vector_df

(4, 2048)


,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
GOYANGDEOGYANG B.png,0.332150,0.013491,0.002084,0.000000,0.013252,0.000000,0.012166,0.569011,0.029282,0.010285,...,0.568901,0.501592,0.0,0.281769,1.534929,0.000000,0.264857,0.005286,0.009107,0.458974
KoPubWorld Batang Light.png,0.846692,0.022705,0.011549,0.000000,0.044180,0.010464,0.000000,1.157957,0.000000,0.049401,...,0.931368,0.951681,0.0,0.185880,1.322598,0.002522,0.336657,0.000000,0.024711,0.899117
KoPubWorld Dotum Medium.png,0.500069,0.046055,0.004663,0.002519,0.016230,0.000000,0.000000,0.494331,0.000000,0.070875,...,0.719389,0.201338,0.0,0.455181,1.051383,0.003287,0.241852,0.016651,0.000000,0.419779
netmarbleM.png,0.637927,0.007158,0.033357,0.000000,0.031003,0.005312,0.000000,0.752176,0.006617,0.000000,...,0.629411,0.752928,0.0,0.238277,1.358989,0.000000,0.346144,0.004403,0.008474,0.550455


### 5.3 입력이미지 - 대표폰트 렌더링 이미지의 유사도 비교

In [ ]:
rendering_1st_vec_sim = []
for i in range(len(rendering_1st_vector_df)):
  rendering_1st_vec_sim.append(cosine_similarity(test_img_vector_df.values, rendering_1st_vector_df.iloc[[i]].values)[0][0])

rendering_1st_vector_sim_df = rendering_1st_vector_df
rendering_1st_vector_sim_df['similarity'] = rendering_1st_vec_sim

### 5.4 대표폰트 중 입력이미지와 가장 유사한 폰트 도출

In [15]:
rendering_1st_vector_sim_df.reset_index(drop=False, inplace=True)
rendering_1st_vector_sim_df
most_sim_font_1st = rendering_1st_vector_sim_df.iloc[[rendering_1st_vector_df['similarity'].argmax()]].index[0]
most_sim_font_1st

3

-> 클러스터 3의 폰트가 입력이미지와 가장 유사함

## 6. 2차 렌더링

### 6.1 가장 유사한 대표폰트가 해당된 클러스터의 폰트로 렌더링

In [17]:
font_list_2nd_rendering = fonts_foreach_cluster.iloc[:, 3].dropna().to_numpy()

most_sim_cluster_font_list = []
for font in font_list_2nd_rendering:
  most_sim_cluster_font_list.append(font + '.ttf')

rendering_time = 2
trim_img_path = make_image(msg, rendering_time, most_sim_cluster_font_list, img_save_path)

trim(trim_img_path)

### 6.2 렌더링한 텍스트 이미지의 특징벡터 추출

In [18]:
input_path = trim_img_path
list_pics2 = []
filenames2 = []
for file2 in os.listdir(input_path):
    filename2 = os.fsdecode(file2)
    img2 = Image.open(os.path.join(input_path, filename2)).convert('RGB')
    list_pics2.append(img2)
    filenames2.append(filename2)
    
vectors2 = img2vec.get_vec(list_pics2)
pics2 = {}
for i2, vec2 in enumerate(vectors2):
    pics2[filenames2[i2]] = vec2
    
rendering_2st_vector_df = pd.DataFrame(pics2).transpose()
rendering_2st_vector_df

(33, 2048)


,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
20950870_경기천년바탕TTF_Bold.png,0.766063,0.079431,0.058991,0.000000,0.068645,0.000000,0.000000,0.880724,0.006750,0.012006,...,0.724126,1.002140,0.000000,0.348568,1.437855,0.006475,0.353435,0.011372,0.031972,0.947058
20950871_경기천년바탕TTF_Regular.png,0.858168,0.050419,0.085935,0.000000,0.056914,0.000000,0.000000,0.559032,0.003958,0.049439,...,0.836833,0.761439,0.000000,0.377806,1.170852,0.009225,0.381820,0.003828,0.163740,0.654001
20950938_Batang_TTF.png,0.471761,0.046428,0.038034,0.000000,0.034413,0.000000,0.000000,0.846828,0.000000,0.040101,...,0.574298,0.336389,0.000000,0.166869,1.367407,0.000000,0.206918,0.000520,0.034196,0.814739
20956073_SuncheonB_TTF.png,0.637555,0.045405,0.043697,0.000000,0.061356,0.005669,0.000000,0.451915,0.008449,0.000000,...,0.713291,0.743179,0.000000,0.174313,1.294787,0.000000,0.402910,0.001051,0.054167,0.386119
20958935_UTOIMAGE_유토이미지고딕_R_TTF.png,0.638247,0.024801,0.018880,0.000000,0.054204,0.000000,0.000000,0.480336,0.000000,0.000000,...,0.555086,0.558511,0.000000,0.242512,1.341264,0.000000,0.207876,0.019865,0.000843,0.490320
20958981_UTOIMAGE_유토이미지고딕_B_TTF.png,0.663762,0.020827,0.000000,0.000000,0.025774,0.010723,0.000000,0.572396,0.000175,0.000000,...,0.452061,0.513230,0.000000,0.207343,1.486952,0.000000,0.282575,0.024125,0.011709,0.451474
20958983_UTOIMAGE_유토이미지고딕_L_TTF.png,0.690835,0.050920,0.042378,0.000000,0.044100,0.000000,0.000000,0.341726,0.000000,0.105523,...,0.654142,0.311590,0.000000,0.420132,0.897241,0.000609,0.252075,0.039092,0.001132,0.469673
BinggraeⅡ.png,0.771842,0.061531,0.012745,0.000000,0.034225,0.000000,0.000000,0.563794,0.004149,0.013339,...,0.615646,0.547340,0.000000,0.028025,1.037422,0.000000,0.352876,0.004138,0.004067,0.972105
BMJUA_ttf.png,0.798613,0.010285,0.060561,0.000000,0.000000,0.004118,0.000000,0.634541,0.000000,0.014777,...,0.303012,0.522438,0.000000,0.149000,1.093477,0.000000,0.744288,0.004317,0.128793,0.482913
DXBrmRExtraBold-KSCpc-EUC-H.png,0.615551,0.013255,0.000000,0.000000,0.000000,0.051375,0.013683,0.760977,0.001498,0.007061,...,0.682078,0.554563,0.000000,0.323130,1.347324,0.000000,0.383690,0.054874,0.024772,0.612431


### 6.3 입력이미지 - 클러스터폰트 렌더링 이미지의 유사도 비교

In [19]:
rendering_2st_vec_sim = []
for i in range(len(rendering_2st_vector_df)):
  rendering_2st_vec_sim.append(cosine_similarity(test_img_vector_df.values, rendering_2st_vector_df.iloc[[i]].values)[0][0])

rendering_2st_vector_sim_df = rendering_2st_vector_df
rendering_2st_vector_sim_df['similarity'] = rendering_2st_vec_sim
rendering_2st_vector_sim_df

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,similarity
20950870_경기천년바탕TTF_Bold.png,0.766063,0.079431,0.058991,0.000000,0.068645,0.000000,0.000000,0.880724,0.006750,0.012006,...,1.002140,0.000000,0.348568,1.437855,0.006475,0.353435,0.011372,0.031972,0.947058,0.802969
20950871_경기천년바탕TTF_Regular.png,0.858168,0.050419,0.085935,0.000000,0.056914,0.000000,0.000000,0.559032,0.003958,0.049439,...,0.761439,0.000000,0.377806,1.170852,0.009225,0.381820,0.003828,0.163740,0.654001,0.796573
20950938_Batang_TTF.png,0.471761,0.046428,0.038034,0.000000,0.034413,0.000000,0.000000,0.846828,0.000000,0.040101,...,0.336389,0.000000,0.166869,1.367407,0.000000,0.206918,0.000520,0.034196,0.814739,0.767940
20956073_SuncheonB_TTF.png,0.637555,0.045405,0.043697,0.000000,0.061356,0.005669,0.000000,0.451915,0.008449,0.000000,...,0.743179,0.000000,0.174313,1.294787,0.000000,0.402910,0.001051,0.054167,0.386119,0.833821
20958935_UTOIMAGE_유토이미지고딕_R_TTF.png,0.638247,0.024801,0.018880,0.000000,0.054204,0.000000,0.000000,0.480336,0.000000,0.000000,...,0.558511,0.000000,0.242512,1.341264,0.000000,0.207876,0.019865,0.000843,0.490320,0.805552
20958981_UTOIMAGE_유토이미지고딕_B_TTF.png,0.663762,0.020827,0.000000,0.000000,0.025774,0.010723,0.000000,0.572396,0.000175,0.000000,...,0.513230,0.000000,0.207343,1.486952,0.000000,0.282575,0.024125,0.011709,0.451474,0.802966
20958983_UTOIMAGE_유토이미지고딕_L_TTF.png,0.690835,0.050920,0.042378,0.000000,0.044100,0.000000,0.000000,0.341726,0.000000,0.105523,...,0.311590,0.000000,0.420132,0.897241,0.000609,0.252075,0.039092,0.001132,0.469673,0.784218
BinggraeⅡ.png,0.771842,0.061531,0.012745,0.000000,0.034225,0.000000,0.000000,0.563794,0.004149,0.013339,...,0.547340,0.000000,0.028025,1.037422,0.000000,0.352876,0.004138,0.004067,0.972105,0.806416
BMJUA_ttf.png,0.798613,0.010285,0.060561,0.000000,0.000000,0.004118,0.000000,0.634541,0.000000,0.014777,...,0.522438,0.000000,0.149000,1.093477,0.000000,0.744288,0.004317,0.128793,0.482913,0.797806
DXBrmRExtraBold-KSCpc-EUC-H.png,0.615551,0.013255,0.000000,0.000000,0.000000,0.051375,0.013683,0.760977,0.001498,0.007061,...,0.554563,0.000000,0.323130,1.347324,0.000000,0.383690,0.054874,0.024772,0.612431,0.871628


### 6.4 입력이미지와 가장 유사한 폰트 및 유사 폰트 추천 순위 도출

In [20]:
most_sim_font_2st = rendering_2st_vector_sim_df.iloc[[rendering_2st_vector_df['similarity'].argmax()]]
print('입력이미지와 가장 유사한 폰트 : ', most_sim_font_2st.index[0].replace('.png', ''))

### 추천 폰트 순위
rendering_2st_vector_sim_df.sort_values(by=['similarity'],ascending=False)

입력이미지와 가장 유사한 폰트 :  DXBrmRExtraBold-KSCpc-EUC-H


,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,similarity
DXBrmRExtraBold-KSCpc-EUC-H.png,0.615551,0.013255,0.000000,0.000000,0.000000,0.051375,0.013683,0.760977,0.001498,0.007061,...,0.554563,0.000000,0.323130,1.347324,0.000000,0.383690,0.054874,0.024772,0.612431,0.871628
타이포도담체.png,0.857519,0.048458,0.040303,0.000000,0.016022,0.050816,0.030369,1.028559,0.045356,0.009049,...,0.938585,0.008661,0.106020,1.495130,0.001692,0.505025,0.012167,0.037842,0.959857,0.840308
20956073_SuncheonB_TTF.png,0.637555,0.045405,0.043697,0.000000,0.061356,0.005669,0.000000,0.451915,0.008449,0.000000,...,0.743179,0.000000,0.174313,1.294787,0.000000,0.402910,0.001051,0.054167,0.386119,0.833821
netmarbleM.png,0.637927,0.007158,0.033357,0.000000,0.031003,0.005312,0.000000,0.752176,0.006617,0.000000,...,0.752928,0.000000,0.238277,1.358989,0.000000,0.346144,0.004403,0.008474,0.550455,0.831935
netmarbleB.png,0.813060,0.054326,0.012691,0.000000,0.018263,0.057418,0.000000,0.800965,0.040571,0.013626,...,1.131447,0.000000,0.143459,1.298105,0.000000,0.527389,0.024271,0.023466,0.754872,0.829228
OSeongandHanEum-Bold.png,0.522416,0.043869,0.000000,0.002603,0.006858,0.006792,0.000000,0.553388,0.030029,0.012896,...,0.715427,0.000000,0.025475,1.479592,0.000000,0.314840,0.013072,0.051440,0.386714,0.823109
THE명품굴림B.png,0.514389,0.032572,0.000000,0.000000,0.012102,0.014152,0.004273,0.605371,0.003201,0.004257,...,0.285064,0.000000,0.200408,1.616786,0.000000,0.389607,0.006483,0.058491,0.250384,0.822821
DXMgoRB-KSCpc-EUC-H.png,0.598484,0.047455,0.053341,0.000000,0.060422,0.005738,0.001646,0.743697,0.042203,0.009499,...,0.765233,0.000000,0.425670,1.476613,0.002054,0.272966,0.015995,0.015715,0.721337,0.819760
HANSolB.png,0.930196,0.012576,0.005711,0.000000,0.000000,0.012707,0.003734,1.139461,0.014411,0.005937,...,1.072321,0.000000,0.194630,1.227836,0.008862,0.440789,0.030601,0.149141,0.962127,0.808877
HANSolM.png,0.852069,0.001107,0.000000,0.000000,0.004196,0.009013,0.000000,1.125504,0.012168,0.000000,...,1.013746,0.000000,0.254365,1.177223,0.005400,0.395449,0.034519,0.012114,0.843219,0.806447
